# Introduction

This notebook will train a very simple model that will predict the sin of a number.

We'll convert the model into something that can be used by tflite and then run it on the ESP32

In [1]:
import math
import os

import numpy as np
import tensorflow as tf

d:\Anaconda\lib\site-packages\scipy\__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.25.0 is required for this version of SciPy (detected version 1.26.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


## Create a dataset to train our model
We'll create a python generator and feed that through a tensorflow Dataset to train our model

In [2]:
def get_data():
  """
  The code will generate a set of random `x` values,calculate their sine
  values.
  """
  # Generate a uniformly distributed set of random numbers in the range from
  # 0 to 2π, which covers a complete sine wave oscillation
  x_values = np.random.uniform(low=0, high=2 * math.pi,
                               size=1000).astype(np.float32)

  # Shuffle the values to guarantee they're not in order
  np.random.shuffle(x_values)

  # Calculate the corresponding sine values
  y_values = np.sin(x_values).astype(np.float32)

  return (x_values, y_values)

## Create and Compile our model
For our loss function we will use mse.


In [3]:
def create_model() -> tf.keras.Model:
  model = tf.keras.Sequential()

  # First layer takes a scalar input and feeds it through 16 "neurons". The
  # neurons decide whether to activate based on the 'relu' activation function.
  model.add(tf.keras.layers.Dense(16, activation='relu', input_shape=(1, )))

  # The new second and third layer will help the network learn more complex
  # representations
  model.add(tf.keras.layers.Dense(16, activation='relu'))

  # Final layer is a single neuron, since we want to output a single value
  model.add(tf.keras.layers.Dense(1))

  # Compile the model using the standard 'adam' optimizer and the mean squared
  # error or 'mse' loss function for regression.
  model.compile(optimizer='adam', loss='mse', metrics=['mae'])

  return model

## Export our model for tflite
We need to convert our model into a tflite model

In [4]:
def convert_tflite_model(model):
  """Convert the save TF model to tflite model, then save it as .tflite flatbuffer format
    Args:
        model (tf.keras.Model): the trained hello_world Model
    Returns:
        The converted model in serialized format.
  """
  converter = tf.lite.TFLiteConverter.from_keras_model(model)
  tflite_model = converter.convert()
  return tflite_model

In [5]:
def save_tflite_model(tflite_model, model_name):
  """save the converted tflite model
  Args:
      tflite_model (binary): the converted model in serialized format.
      save_dir (str): the save directory
      model_name (str): model name to be saved
  """
  with open(model_name, "wb") as f:
    f.write(tflite_model)

## Train Model

In [6]:
def train_model(epochs, x_values, y_values):
  """Train keras hello_world model
    Args: epochs (int) : number of epochs to train the model
        x_train (numpy.array): list of the training data
        y_train (numpy.array): list of the corresponding array
    Returns:
        tf.keras.Model: A trained keras hello_world model
  """
  model = create_model()
  model.fit(x_values,
            y_values,
            epochs=epochs,
            validation_split=0.2,
            batch_size=64,
            verbose=2)

  return model


# Main

## Testing our model
We can feed in some values and see what our model predicts

In [9]:
def main(_):
  x_values, y_values = get_data()
  trained_model = train_model(500, x_values, y_values)

  test_X = np.array([
    [0.1],
    [0.3],
    [0.5],
    [0.7]
    ])
  Y = trained_model.predict_on_batch(test_X)
  print(Y)

  # Convert and save the model to .tflite
  tflite_model = convert_tflite_model(trained_model)
  save_tflite_model(tflite_model,
                    model_name="converted_model.tflite")

In [10]:
main(_)

Epoch 1/500
13/13 - 1s - loss: 4.1123 - mae: 1.5992 - val_loss: 2.7397 - val_mae: 1.3164 - 645ms/epoch - 50ms/step
Epoch 2/500
13/13 - 0s - loss: 2.3262 - mae: 1.2522 - val_loss: 1.5633 - val_mae: 1.0434 - 45ms/epoch - 3ms/step
Epoch 3/500
13/13 - 0s - loss: 1.3546 - mae: 1.0077 - val_loss: 1.0100 - val_mae: 0.8766 - 50ms/epoch - 4ms/step
Epoch 4/500
13/13 - 0s - loss: 0.9499 - mae: 0.8720 - val_loss: 0.7912 - val_mae: 0.7984 - 54ms/epoch - 4ms/step
Epoch 5/500
13/13 - 0s - loss: 0.7780 - mae: 0.8019 - val_loss: 0.6726 - val_mae: 0.7478 - 57ms/epoch - 4ms/step
Epoch 6/500
13/13 - 0s - loss: 0.6655 - mae: 0.7476 - val_loss: 0.5890 - val_mae: 0.7027 - 54ms/epoch - 4ms/step
Epoch 7/500
13/13 - 0s - loss: 0.5873 - mae: 0.7004 - val_loss: 0.5305 - val_mae: 0.6648 - 48ms/epoch - 4ms/step
Epoch 8/500
13/13 - 0s - loss: 0.5331 - mae: 0.6569 - val_loss: 0.4926 - val_mae: 0.6339 - 45ms/epoch - 3ms/step
Epoch 9/500
13/13 - 0s - loss: 0.5016 - mae: 0.6280 - val_loss: 0.4699 - val_mae: 0.6085 - 46m

INFO:tensorflow:Assets written to: C:\Users\leand\AppData\Local\Temp\tmpsr5s8xev\assets


## To convert to C++
We can then run this command to convert the model to c code.
```
xxd -i converted_model.tflite > model_data.cc
```